In [1]:
import pandas as pd
import numpy as np
import json
import re
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
def load_json_data(file_path):
    try:
        data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in tqdm(f, desc="Carregando dados"):
                try:
                    item = json.loads(line)
                    data.append(item)
                except json.JSONDecodeError:
                    continue
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")
        return None


print("Carregando o arquivo trn.json...")
df = load_json_data("trn.json")



Carregando o arquivo trn.json...


Carregando dados: 0it [00:00, ?it/s]

In [6]:
print("\nInformações do DataFrame:")
print(f"Número de linhas: {df.shape[0]}")
print(f"Número de colunas: {df.shape[1]}")
print("\nColunas disponíveis:")
print(df.columns.tolist())

print("\nPrimeiras 5 linhas do DataFrame:")
print(df.head())

# Verificar se as colunas necessárias existem
required_columns = ['title', 'content']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"\nAtenção: Colunas necessárias não encontradas: {missing_columns}")
    # Verificar nomes de colunas semelhantes que podem ser usadas
    for col in df.columns:
        for req_col in missing_columns:
            if req_col.lower() in col.lower():
                print(f"A coluna '{col}' pode ser usada em vez de '{req_col}'")


Informações do DataFrame:
Número de linhas: 29216
Número de colunas: 5

Colunas disponíveis:
['uid', 'title', 'content', 'target_ind', 'target_rel']

Primeiras 5 linhas do DataFrame:
          uid                                                           title  \
0  0000031909                                     Girls Ballet Tutu Neon Pink   
1  0000032034                                        Adult Ballet Tutu Yellow   
2  0000913154  The Way Things Work: An Illustrated Encyclopedia of Technology   
3  0001360000                                                   Mog's Kittens   
4  0001381245                                           Misty of Chincoteague   

                                                                                                                                                                                                                                                                                                                                         

Verificar valores nulos

In [7]:

print("\nValores nulos por coluna:")
print(df[required_columns].isnull().sum())


Valores nulos por coluna:
title      0
content    0
dtype: int64


In [9]:
def preprocess_data(df, title_col='title', content_col='content'):
    # Criar uma cópia para não modificar o original
    processed_df = df.copy()

    # Garantir que as colunas existam
    if title_col not in processed_df.columns or content_col not in processed_df.columns:
        print(f"Erro: Colunas {title_col} e/ou {content_col} não encontradas.")
        return None

    # Converter para string e tratar nulos
    processed_df[title_col] = processed_df[title_col].fillna("").astype(str)
    processed_df[content_col] = processed_df[content_col].fillna("").astype(str)

    # Remover linhas com título ou conteúdo vazios
    empty_count = len(processed_df[(processed_df[title_col].str.strip() == "") |
                                   (processed_df[content_col].str.strip() == "")])
    print(f"Removendo {empty_count} linhas com título ou conteúdo vazios...")
    processed_df = processed_df[(processed_df[title_col].str.strip() != "") &
                                (processed_df[content_col].str.strip() != "")]

    # Limpeza básica
    print("Limpando os dados...")

    def clean_text(text):
        # Remove caracteres especiais indesejados e espaços extras
        text = re.sub(r'\s+', ' ', text)  # Espaços múltiplos
        text = text.strip()  # Espaços no início e final
        return text

    processed_df[title_col] = processed_df[title_col].apply(clean_text)
    processed_df[content_col] = processed_df[content_col].apply(clean_text)

    # Remover duplicatas
    duplicate_count = processed_df.duplicated(subset=[title_col, content_col]).sum()
    print(f"Removendo {duplicate_count} entradas duplicadas...")
    processed_df = processed_df.drop_duplicates(subset=[title_col, content_col])

    print(f"Pré-processamento concluído. Restaram {len(processed_df)} exemplos.")
    return processed_df

cleaned_df = preprocess_data(df)

Removendo 6905 linhas com título ou conteúdo vazios...
Limpando os dados...
Removendo 173 entradas duplicadas...
Pré-processamento concluído. Restaram 22138 exemplos.


In [11]:
def prepare_prompts_for_finetuning(df, title_col='title', content_col='content'):
    """
    Prepara os prompts para fine-tuning baseado no tipo de modelo.

    Args:
        df: DataFrame com os dados
        title_col: Nome da coluna do título
        content_col: Nome da coluna do conteúdo

    Returns:
        DataFrame com os prompts formatados
    """
    prompts_df = df.copy()

    prompts_df['prompt'] = "Gere uma descrição detalhada para um produto com o título: " + prompts_df[title_col]
    prompts_df['completion'] = prompts_df[content_col]

    return prompts_df


# Preparar os prompts
prompts_df = prepare_prompts_for_finetuning(cleaned_df)

for i, row in prompts_df.head(3).iterrows():
    print("\n" + "=" * 50)
    print(f"Prompt: {row['prompt']}")
    print(f"Completion: {row['completion'][:100]}...")


Prompt: Gere uma descrição detalhada para um produto com o título: Girls Ballet Tutu Neon Pink
Completion: High quality 3 layer ballet tutu. 12 inches in length...

Prompt: Gere uma descrição detalhada para um produto com o título: Mog's Kittens
Completion: Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have ...

Prompt: Gere uma descrição detalhada para um produto com o título: Girls Ballet Tutu Neon Blue
Completion: Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes...


In [12]:
def export_data_for_finetuning(df, train_ratio=0.95, output_dir='finetuning_data'):
    """
    Divide os dados em treino e validação e exporta no formato necessário para fine-tuning.

    Args:
        df: DataFrame com os dados formatados
        train_ratio: Proporção de dados para treino
        output_dir: Diretório para salvar os arquivos
    """
    # Criar diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Dividir os dados em treino e validação
    train_df, val_df = train_test_split(df, train_size=train_ratio, random_state=42)

    print(f"Dados divididos: {len(train_df)} exemplos para treino, {len(val_df)} exemplos para validação.")

    # Formato geral para outros modelos (prompt + completion)
    train_path = os.path.join(output_dir, 'train_llama.jsonl')
    val_path = os.path.join(output_dir, f'val_llama.jsonl')

    columns_to_export = ['prompt', 'completion'] if 'prompt' in df.columns else df.columns

    with open(train_path, 'w', encoding='utf-8') as f:
        for _, row in train_df.iterrows():
            export_row = {col: row[col] for col in columns_to_export}
            f.write(json.dumps(export_row) + '\n')

    with open(val_path, 'w', encoding='utf-8') as f:
        for _, row in val_df.iterrows():
            export_row = {col: row[col] for col in columns_to_export}
            f.write(json.dumps(export_row) + '\n')

    print(f"Dados exportados para:")
    print(f"- Treino: {train_path}")
    print(f"- Validação: {val_path}")

    # Salvar alguns exemplos para referência rápida
    examples_path = os.path.join(output_dir, f'examples_llama.json')
    with open(examples_path, 'w', encoding='utf-8') as f:
        examples = train_df.head(5).to_dict('records')
        json.dump(examples, f, indent=2, ensure_ascii=False)

    print(f"- Exemplos: {examples_path}")

    return train_path, val_path


# Exportar os dados
train_path, val_path = export_data_for_finetuning(prompts_df)

Dados divididos: 21031 exemplos para treino, 1107 exemplos para validação.
Dados exportados para:
- Treino: finetuning_data/train_llama.jsonl
- Validação: finetuning_data/val_llama.jsonl
- Exemplos: finetuning_data/examples_llama.json
